 <h2> Domain.com.au Web Scraping</h2>
                             <img src="http://www.domain.com.au/Content/Files/misc/domainLogoHead.gif">

IMPORTING LIBRARIES

In [ ]:
# HTML extraction & Parsin
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import urllib
import math
import os
from constants import Brisbane_suburbs,Melbourne_suburbs,Sydney_suburbs,Adelaide_suburbs,Perth_suburbs



# Calculate Latitude & Longitude
from geopy.geocoders import Nominatim
# Calculate Distance from Latitude & Longitude
from math import sin, cos, sqrt, atan2, radians
import emailsend



SET GLOBAL PARAMETERS

In [ ]:
urls = []
# Change the index to change the city
city_list = ['Sydney','Melbourne','Brisbane','Adelaide','Perth']
city = city_list[3]
if city == 'Sydney':
    suburbs_list = Sydney_suburbs
elif city == 'Melbourne':
    suburbs_list = Melbourne_suburbs
elif city == 'Brisbane':
    suburbs_list = Brisbane_suburbs
elif city == 'Adelaide':
    suburbs_list = Adelaide_suburbs
elif city == 'Perth':
    suburbs_list = Perth_suburbs



print(f"CHOSEN CITY: {city}")
if not os.path.exists(f"../MLrealestate/data/{city}/"):
    os.mkdir(f"../MLrealestate/data/{city}/")
print(suburbs_list)
for suburb in suburbs_list:
    urls.append(f"https://www.domain.com.au/sold-listings/{suburb}/?excludepricewithheld=1&landsize=10-any&ssubs=0&page=")

house_list = []



API CALL VIA PROXY

In [ ]:
for url in urls:
    max_pages = 50
    for x in range(1,max_pages+1):
        full_url = 'https://api.proxycrawl.com/?token=QviEcDJvtDVu2Jwa2GwbiQ&url=' +urllib.parse.quote_plus(url+str(x))
        print(full_url)
        html_text = requests.get(full_url).text
        soup = BeautifulSoup(html_text,'lxml')
        if x ==1:
            try:
                max_pages = math.ceil(int(soup.find(attrs = {'data-testid':'summary'}).text.split(' ')[0])/20)
                print(str(max_pages) + ' is the number of pages found')
                if max_pages > 50:
                    print(str(max_pages)+ ' is the number of max pages found but we are capping it to 50')
                    max_pages = 50
            except:
                max_pages = 2
        if x > max_pages:
            break
                
            
        houses = soup.find_all('li',class_= 'css-1qp9106')

        for house in houses:

            # Extract Data Sold & Method of Sale
            date_method = house.find('span', class_='css-1nj9ymt').text
            date_method_split = date_method.split(" ")
            date = f"{date_method_split[-3]} {date_method_split[-2]} {date_method_split[-1]}"
            if "auction" in date_method:
                method = 'auction'
            elif "private treaty" in date_method:
                method = 'private treaty'
            else:
                method = 'Another Method'

            # Extract Address details
            try:
    
                street = house.find('span', class_='css-iqrvhs',attrs={'data-testid': 'address-line1'}).text.replace(',','')
                
            except:
                print('ISSUE FOUND WITH STREET')
                street = 'No Street Found'
            
            try:
                 address = house.find('span', class_='css-iqrvhs',attrs={'data-testid': 'address-line2'}).text
                 address_split = address.split(' ')
                 postcode = address_split[-1]
                 state = address_split[-2]
                 if len(address_split) == 4:
                    suburb = f"{address_split[0]} {address_split[1]}"
                 elif len(address_split) == 5:
                    suburb = f"{address_split[0]} {address_split[1]} {address_split[2]}"
                 else:
                    suburb = address_split[0]
                
            except:
                print('ISSUE FOUND WITH ADDRESS')
            
               
            

            
            price = house.find('p', class_='css-mgq8yx').text.replace(' price from APM PriceFinder','').replace('$','').replace(',','')
            type = house.find('span', class_='css-693528').text.replace('/ Unit / Flat','')

            try:
                features = house.find('div', class_ ='css-18biwo', attrs = {'data-testid':'property-features-wrapper'})
                bedrooms = features.span.span.text[0]
                if bedrooms == '−' or bedrooms is None:
                    bedrooms = 0
                bathrooms = features.span.next_sibling.span.text[0]
                if bathrooms == '−' or bathrooms is None:
                    bathrooms = 1
                cars = features.span.next_sibling.next_sibling.span.text[0]
                if cars == '−':
                    cars = 0
                if len(list(features.children)) == 4:
                    area = features.span.next_sibling.next_sibling.next_sibling.span.text
                else:
                    area = 0
            except:
                bedrooms = 0
                bathrooms = 0
                cars = 0
                area = 0
                print(f"No features found for {street} {address}")

            agent = house.find('div', class_='css-6yavch',attrs={'data-testid':'listing-card-lazy-image'})
            if agent is None:
                agent = 'UNKNOWN'
            elif agent.img['alt'][0] == 'P':
                agent = 'UNKNOWN'
            else:
                agent = agent.img['alt'].replace('Logo for ','')
            
            
            
        
            house_details = {
                'Street' : street,
                'Address' : address,
                'Suburb' : suburb,
                'State' : state,
                'Postcode' : postcode,
                'Type' : type,
                'Date': date,
                'Method' : method,
                'Price' : price,
                'Bedrooms': bedrooms,
                'Bathrooms': bathrooms,
                'Cars' : cars,
                'Area' : area,
                'Agent' : agent
            }
            if (bedrooms == 0 and bathrooms == 0 and cars == 0 and area == 0) or street == 'No Street Found':
                print(f"{street} {address} house was NOT added")
            else:
                house_list.append(house_details)
               
        print(str(len(house_list))+' houses added so far')
        if len(house_list) > 1000:
            df = pd.DataFrame(house_list)
            df.to_csv(f"../MLrealestate/data/{city}/{city}_raw.csv",mode='a', header = False, index = False)
            house_list = []



SAVING PANDAS DF TO CSV. PLEASE NOTE: WE ARE APPENDING TO CSV

In [ ]:


df = pd.DataFrame(house_list)
df.to_csv(f"../MLrealestate/data/{city}/{city}_raw.csv",mode='a', header = True, index = False)
df.tail(6)

Creating Latitude & Longitude columns from address

In [ ]:

df = pd.read_csv(f"../MLrealestate/data/{city}/{city}_raw.csv")

def get_coordinates(street, suburb):
    loc = Nominatim(user_agent="GetLoc")

    try:
        if {street} == 'No Street Found':
            getLoc = loc.geocode(f"{suburb},Australia")
        else:
            getLoc = loc.geocode(f"{street},{suburb},Australia")
        lat = getLoc.latitude
        long = getLoc.longitude
    except:
        try:
            getLoc = loc.geocode(f"{suburb},Australia")
            lat = getLoc.latitude
            long = getLoc.longitude
        except:
            print(f"{street},{suburb},Australia was not able to find lat & long")
            lat = -37.7618477
            long = 144.9805318
    return lat,long


vect_get_coordinates = np.vectorize(get_coordinates)



for i in range(0,len(df.index),1000):
    df_seg = df[i:i+1000]
    df_seg['Latitude'], df_seg['Longitude']  = vect_get_coordinates(df_seg.Street, df_seg.Address)
    df_seg.to_csv(f"../MLrealestate/data/{city}/{city}_latlon.csv",mode='a',header = False,index=False)
    print(f"PROCESSED: {len(df_seg)} RECORDS OUT OF TOTAL {len(df.index)}")
    print(f"PROGRESS: {len(df_seg)/len(df.index)} % COMPLETED")

    

CALCULATE DISTANCE FROM CITY by vectorization function and using Latitude and longitude

In [ ]:


df = pd.read_csv(f"../data/{city}/{city}_latlon.csv")

suburb_of_city = f"{city.split('-')[0]}"
la2,lo2 = get_coordinates('No Street Found', suburb_of_city)


def calc_distance(la1,lo1):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(la1)
    lon1 = radians(lo1)
    lat2 = radians(la2)
    lon2 = radians(lo2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance
   

vec_cal_distance = np.vectorize(calc_distance)
df['Distance']  = vec_cal_distance(df.Latitude, df.Longitude)
df.to_csv(f"../data/{city}/{city}_distance.csv",header = True,index=False)
df.head(20)

Clean up Area by removing commas,m2, and converting Ha to m2

In [ ]:
df = pd.read_csv(f"../MLrealestate/data/{city}/{city}_distance.csv")

def convert_area(area):
    if ',' in area:
        area = area.replace(',','')
    if 'm²' in area:
        area = area.replace('m²','')
    if 'ha' in area:
        area = 10000*float(area.replace('ha',''))
    return float(area)
   

vec_convert_area = np.vectorize(convert_area)
df['Area']  = vec_convert_area(df.Area)


df.to_csv(f"../MLrealestate/data/{city}/{city}_area.csv",header = True,index=False)

Send Email of the collected & transformed data

In [ ]:

emailsend.run(f"../MLrealestate/data/{city}/{city}_distance.csv")